In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
split_lines = [l.split() for l in lines]
dig_plan = [(l[0], int(l[1]), l[2]) for l in split_lines]
dig_plan[:5]

In [ ]:
def mv_pos(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def print_dig_field(visited_fields, max_x, max_y):
    for i in range(0, max_x):
        for j in range(0, max_y):
            if (i, j) in visited_fields:
                print("#", end="")
            else:
                print(".", end="")
        print()

In [ ]:
dirs = {"R": (0, 1), "U": (-1, 0), "D": (1, 0), "L": (0, -1)}

In [ ]:
start_point = (0, 0)
digged_fields = []
current_dig_field = start_point

for d in dig_plan:
    for i in range(0, d[1]):
        digged_fields.append(current_dig_field)
        current_dig_field = mv_pos(current_dig_field, dirs[d[0]])
assert current_dig_field == start_point

min_x_coord = min([x[0] for x in list(digged_fields)])
min_y_coord = min([x[1] for x in list(digged_fields)])
max_x_coord = max([x[0] for x in list(digged_fields)])
max_y_coord = max([x[1] for x in list(digged_fields)])

# print(f"({min_x_coord},{min_y_coord})")

shifted_fields = [(x[0] - min_x_coord, x[1] - min_y_coord) for x in digged_fields]
shifted_min = (0, 0)
shifted_max = (max_x_coord - min_x_coord + 1, max_y_coord - min_y_coord + 1)

# print_dig_field(
#    shifted_fields, shifted_max[0], shifted_max[1]
# )

next_lower_dict = {}
for sf in shifted_fields:
    if (sf[0] + 1, sf[1]) in shifted_fields:
        next_lower_dict[sf] = True
    else:
        next_lower_dict[sf] = False
# next_lower_dict

In [ ]:
filled_fields = set()
inside = False
for i in range(0, shifted_max[0]):
    for j in range(0, shifted_max[1]):
        # print(f"({i},{j}) - {inside}")
        if next_lower_dict.get((i, j), False):
            inside = not inside
        elif inside:
            filled_fields.add((i, j))


# print_dig_field(
#    set(shifted_fields).union(filled_fields), shifted_max[0], shifted_max[1]
# )

In [ ]:
len(set(shifted_fields).union(filled_fields))

part 2

In [ ]:
instr_map = {"0": "R", "1": "D", "2": "L", "3": "U"}
hex_lines = [l.split()[-1].replace("(", "").replace(")", "") for l in lines]
new_instr = [(instr_map[hl[-1]], int(hl[1:6], 16)) for hl in hex_lines]
new_instr[:5]

In [ ]:
dig_plan = new_instr